This notebook best viewed here: https://nbviewer.jupyter.org

This notebook is for the interior variety of Douglas fir GEA mapping pops

#### Purpose
• I've run the baypass commands across four servers,

#### Notebook outline

- __rsync files from servers__
    - I execute the rsync commands to transfer files from the executing servers to our lab server
- __check what needs to be done__
    - given the number of gfiles and chains, make sure all of the commands have been executed/transferred
- __gather all of the wanted resfiles and concat into single dataframes__
    - gather the files with BF, label SNP IDs, and concat into single dataframe

In [1]:
from pythonimports import *

In [2]:
baydir = '/lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/coastal_variety/03_maf-p05_RD-recalculated_FDC/baypass'
cmddir = op.join(baydir, 'cmd_files')
rundir = op.join(cmddir, 'run_01')
gfile_dir = op.join(baydir, 'individual_gfiles')
shdir = op.join(cmddir, 'shfiles')
resdir = op.join(cmddir, 'results')
rsyncdir = op.join(baydir, 'rsync_files')

# rsync files from servers

In [3]:
lview,dview = get_client()

56 56


In [40]:
fs(rsyncdir)

['/lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/coastal_variety/03_maf-p05_RD-recalculated_FDC/baypass/rsync_files/arc_rsync_cmds.txt',
 '/lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/coastal_variety/03_maf-p05_RD-recalculated_FDC/baypass/rsync_files/beluga_rsync_cmds.txt',
 '/lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/coastal_variety/03_maf-p05_RD-recalculated_FDC/baypass/rsync_files/cedar_rsync_cmds.txt',
 '/lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/coastal_variety/03_maf-p05_RD-recalculated_FDC/baypass/rsync_files/graham_rsync_cmds.txt']

In [4]:
def exe(cmd):
    import os
    os.system(cmd)

In [5]:
def submit_cmds(cmds):
    jobs = []
    for cmd in nb(cmds):
        jobs.append(lview.apply_async(exe, cmd))
    watch_async(jobs)
    
    badcmds = []
    for i,j in enumerate(jobs):
        try:
            x=j.r
        except:
            badcmds.append(i)
    print('len(badcmds) = ', len(badcmds))
    
    return badcmds

In [6]:
def check_realfiles(cmds):
    realfiles = []
    for d in [rundir, gfile_dir, shdir, resdir]:
        realfiles.extend(fs(d, dirs=False))
    
    expfiles = []
    for cmd in cmds:
        expfiles.append(cmd.split()[-1])
    
    print('len(realfiles) = ', len(realfiles))
    print('len(expfiles) =', len(expfiles))
    
    needed = list(set(expfiles) - set(realfiles))
    
    print('needed = ', len(needed))
    
    needed_cmds = []
    for n in nb(needed):
        for cmd in cmds:
            if n in cmd:
                needed_cmds.append(cmd)
    print('needed_cmds = ', len(needed_cmds))
    return needed_cmds

#### cedar cmds

In [8]:
# first from cedar
cmds = read(op.join(rsyncdir, 'cedar_rsync_cmds.txt'))
len(cmds)

11589

In [42]:
dicouts = [f for f in cmds if 'DIC.out' in f]
len(dicouts)

1872

In [10]:
cmds[-1]

'rsync -avz cedar:/scratch/lindb/coastal/baypass/cmd_files/run_01/catfile_0106.txt /lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/coastal_variety/03_maf-p05_RD-recalculated_FDC/baypass/cmd_files/run_01/catfile_0106.txt'

In [11]:
badcmds = submit_cmds(cmds)

11589
11589
len(badcmds) =  0


In [16]:
needed = check_realfiles(cmds)
needed

len(realfiles) =  11589
len(expfiles) = 11589
needed =  0
needed_cmds =  0


[]

#### arc cmds

In [9]:
arccmds = read(op.join(rsyncdir, 'arc_rsync_cmds.txt'))
len(arccmds)

3845

In [44]:
arcdicouts = [f for f in arccmds if 'DIC.out' in f]
len(arcdicouts)

336

#### beluga cmds

In [10]:
belugacmds = read(op.join(rsyncdir, 'beluga_rsync_cmds.txt'))
len(belugacmds)

5893

In [48]:
bdicouts = [f for f in belugacmds if 'DIC.out' in f]
len(bdicouts)

742

#### graham cmds

In [11]:
gcmds = read(op.join(rsyncdir, 'graham_rsync_cmds.txt'))
len(gcmds)

11589

In [52]:
gdicouts = [f for f in gcmds if 'DIC.out' in f]
len(gdicouts)

1872

In [73]:
thecmds = [cmd for cmd in arccmds + gcmds if not 'individual_gfiles' in cmd]
len(cmds), len(arccmds + gcmds)

(11589, 15434)

In [78]:
len([cmd for cmd in thecmds if cmd.endswith('.out')])

11086

In [81]:
lview,dview = get_client()

56 56


In [82]:
rem = [cmd for cmd in thecmds if not op.exists(cmd.split()[-1])]
len(rem), len(thecmds)

(9360, 11346)

In [83]:
# badcmds = submit_cmds(cmds + belugacmds + gcmds)
# thecmds = [cmd for cmd in arccmds + gcmds if not 'individual_gfiles' in cmd]
badcmds = submit_cmds(rem)

9360
9360
len(badcmds) =  0


In [12]:
needed = []
for cmd in nb(cmds + arccmds + belugacmds + gcmds):
    file = op.realpath(cmd.split()[-1])
    if not op.exists(file):
        needed.append(cmd)
len(needed)

100%|██████████| 32916/32916 [00:02<00:00, 11343.03it/s]


0

In [86]:
badcmds = submit_cmds(needed)

3768
3768
len(badcmds) =  0


#### remaining arc and beluga cmds - these are jobs that didn't finish or were missing

In [7]:
arccmds = read(op.join(rsyncdir, 'arc_rsync_cmds_remaining.txt'))
len(arccmds)

474

In [8]:
belugacmds = read(op.join(rsyncdir, 'beluga_rsync_cmds_remaining.txt'))
len(belugacmds)

1452

In [9]:
allrem = arccmds + belugacmds
len(allrem)

1926

In [10]:
badcmds = submit_cmds(allrem)

1926
1926
len(badcmds) =  0


In [11]:
needed = []
for cmd in nb(allrem):
    file = op.realpath(cmd.split()[-1])
    if not op.exists(file):
        needed.append(cmd)
len(needed)

100%|██████████| 1926/1926 [00:00<00:00, 10626.24it/s]


0

# redo everything below here

# check what needs to be done
given the number of gfiles and chains, make sure all of the commands have been executed/transferred

#### first check to make sure every sh file was exececuted (ie, check that each has a slurm .out file)

In [148]:
# the the shfiles and out files
shfiles = fs(shdir, endswith='.sh')
outs = fs(shdir, endswith='.out')
len(shfiles), len(outs)

(107, 107)

In [13]:
# which sh files were not run, print out any that weren't run
pairs = {}
for sh in shfiles:
    pairs[sh] = []
    for out in outs:
        if op.basename(sh).split(".")[0] in out:
            pairs[sh].append(out)
for sh,outfiles in pairs.items():
    if len(outfiles) != 1:
        print(sh, len(outfiles))

In [15]:
# go and run these jobs ^ on beluga

#### go through catfiles and make sure all of the commands inside were run

I expect at least one command to have not been executed, and several to have failed on graham

In [14]:
resdir

'/lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/coastal_variety/03_maf-p05_RD-recalculated_FDC/baypass/cmd_files/results'

In [15]:
# get one of the files for each of the commands that executed successfully (count the succesful cmds)
resfiles = fs(resdir, endswith='DIC.out')
len(resfiles)

5110

In [16]:
# get the files with the baypass commands - this matches number of sh files (good)
catfiles = fs(rundir, pattern='catfile')
len(catfiles)

107

In [17]:
# iterate through catfile commands and make sure there's a result for each
found = {}  # map prefix to output file
exe = {}  # map prefix to command used
for cat in nb(catfiles):
    # get the commands
    cmds = read(cat)
    
    # assign prefixes (= num cmds) to their output
    for cmd in cmds:
        prefix = cmd.split("-outprefix")[1].split()[0]
        assert prefix not in found and 'chain' in prefix
        exe[prefix] = cmd
        found[prefix] = []
        for res in resfiles:
            if prefix in op.basename(res):
                found[prefix].append(res)
prefix

100%|██████████| 107/107 [00:20<00:00,  5.25it/s]


'interior_all_snps_ADP-lt-1000_gfile_1021_chain_5'

In [95]:
# figure out which prefixes have zero output (and need to be reran) or cmds with >1 output (impossible...)
needed_cmds = []
for prefix,reslist in found.items():
    if len(reslist) == 0:
        needed_cmds.append(exe[prefix])  # get the cmd that needs to be executed
    elif len(reslist) > 1:
        print('\n', prefix, len(reslist))  # this shouldn't be possible, check anyway, print if so
len(needed_cmds)

288

In [18]:
# after doing needed_cmds on servers (allrem object)
# figure out which prefixes have zero output (and need to be reran) or cmds with >1 output (impossible...)
needed_cmds = []
for prefix,reslist in found.items():
    if len(reslist) == 0:
        needed_cmds.append(exe[prefix])  # get the cmd that needs to be executed
    elif len(reslist) > 1:
        print('\n', prefix, len(reslist))  # this shouldn't be possible, check anyway, print if so
len(needed_cmds)

0

In [19]:
len(found)

5110

# gather all of the wanted resfiles and concat into single dataframes

gather the files with BF, label SNP IDs, label environmental covariables, and concat into single dataframe

In [3]:
lview,dview = get_client()

56 56


In [4]:
bffiles = fs(resdir, endswith='betai_reg.out')
xtxfiles = fs(resdir, endswith='pi_xtx.out')
len(bffiles), len(xtxfiles)

(5110, 5110)

In [5]:
bf = pd.read_table(bffiles[0], delim_whitespace=True)
bf.head()

,COVARIABLE,MRK,M_Pearson,SD_Pearson,BF(dB),Beta_is,SD_Beta_is,eBPis
0,1,1,0.019481,0.143261,-9.818379,0.001109,0.009161,0.044014
1,1,2,0.067087,0.131969,-4.168396,0.005074,0.009358,0.230858
2,1,3,-0.010818,0.142644,-11.520927,-0.001558,0.005309,0.113974
3,1,4,-0.046764,0.117936,-11.884691,-0.001756,0.006781,0.099268
4,1,5,0.021122,0.128426,-11.381520,0.000681,0.007170,0.034198


In [6]:
xtx = pd.read_table(xtxfiles[0], delim_whitespace=True)
xtx.head()

,MRK,M_P,SD_P,M_XtX,SD_XtX,XtXst,log10(1/pval)
0,1,0.974898,0.026716,45.273600,9.453964,29.744166,0.017190
1,2,0.965371,0.044352,45.608332,9.550721,38.958259,0.139882
2,3,0.990501,0.008169,43.936132,9.304743,21.079412,0.000392
3,4,0.957969,0.095533,45.098322,9.342395,54.202269,0.786383
4,5,0.980168,0.010309,44.985152,9.365562,50.326890,0.567376


In [7]:
def label_snps(resfile):
    import os
    import pandas
    # find the gfile associated with resfile
    prefix = os.path.basename(resfile).split("_chain")[0]
    chain = "chain_%s" % os.path.basename(resfile).split(prefix)[1].split("_")[2]
    gfile = os.path.join(gfile_dir, f'{prefix}_HEADERIDX.txt')
    gdf = pandas.read_table(gfile, index_col=0, usecols=[0,1])  # read in rows with SNPIDs, but only one column
    
    # read in the resfile
    df = pandas.read_table(resfile, delim_whitespace=True)
    
    # map MRK column to locus name
    gdict = dict((num+1, locus) for num,locus in enumerate(gdf.index))
    
    # map covariable column to environment ID
    edict = dict((num+1, env) for num,env in enumerate(efile.index))
    
    # split each covariable (env) in to its own file
    df['env'] = df['COVARIABLE'].map(edict)
    envdfs = {chain:{}}
    for env in uni(df['env']):
        envdfs[chain][env] = df[df['env']==env].copy()
        envdfs[chain][env].index = gdf.index.tolist()

    return envdfs
efile = pd.read_table(op.join(baydir, 'coastal_efile_std_HEADERIDX.txt'), index_col=0, usecols=[0,1])
dview['efile'] = efile
dview['uni'] = uni
dview['gfile_dir'] = gfile_dir

In [8]:
jobs = []
for bffile in nb(bffiles):
    jobs.append(lview.apply_async(label_snps, bffile))
watch_async(jobs, sleep=5)

5110
5110


In [9]:
# gather output by environment
envlist = {}
for j in nb(jobs):
    if j.ready():
        chain = keys(j.r)[0]
        if chain not in envlist:
            envlist[chain] = {}
        for env in j.r[chain]:
            if env not in envlist[chain]:
                envlist[chain][env] = []
            envlist[chain][env].append(j.r[chain][env])
keys(envlist)

100%|██████████| 5110/5110 [00:00<00:00, 18691.76it/s]


['chain_1', 'chain_2', 'chain_3', 'chain_4', 'chain_5']

In [10]:
# concatenate environmental output by each chain, print out num snps for each env for each chain
chaindfs = {}
for chain,envs in envlist.items():
    assert len(envs) == 19
    if chain not in chaindfs:
        chaindfs[chain] = {}
    for env,dfs in nb(envs.items(), desc=chain):
        if env not in chaindfs[chain]:
            chaindfs[chain][env] = pd.concat(dfs)
        else:
            print('crap ', chain, env)
for chain in chaindfs:
    print(ColorText(chain).bold())
    for env in chaindfs[chain]:
        print('\t',env, nrow(chaindfs[chain][env]))

chain_5: 100%|██████████| 19/19 [00:08<00:00,  2.17it/s]

chain_1
	 DD_0 2350569
	 MCMT 2350569
	 EMT 2350569
	 FFP 2350569
	 Eref 2350569
	 bFFP 2350569
	 PAS 2350569
	 TD 2350569
	 AHM 2350569
	 EXT 2350569
	 eFFP 2350569
	 SHM 2350569
	 NFFD 2350569
	 MAP 2350569
	 MSP 2350569
	 CMD 2350569
	 MWMT 2350569
	 DD5 2350569
	 MAT 2350569
chain_2
	 PAS 2350569
	 CMD 2350569
	 DD5 2350569
	 MCMT 2350569
	 MWMT 2350569
	 MAT 2350569
	 DD_0 2350569
	 NFFD 2350569
	 eFFP 2350569
	 AHM 2350569
	 EMT 2350569
	 Eref 2350569
	 SHM 2350569
	 EXT 2350569
	 MAP 2350569
	 TD 2350569
	 FFP 2350569
	 MSP 2350569
	 bFFP 2350569
chain_3
	 MCMT 2350569
	 MWMT 2350569
	 EXT 2350569
	 TD 2350569
	 MSP 2350569
	 eFFP 2350569
	 NFFD 2350569
	 bFFP 2350569
	 PAS 2350569
	 DD_0 2350569
	 EMT 2350569
	 MAP 2350569
	 AHM 2350569
	 CMD 2350569
	 Eref 2350569
	 FFP 2350569
	 DD5 2350569
	 MAT 2350569
	 SHM 2350569
chain_4
	 DD5 2350569
	 TD 2350569
	 DD_0 2350569
	 NFFD 2350569
	 EMT 2350569
	 MSP 2350569
	 EXT 2350569
	 MAP 2350569
	 MCMT 2350569
	 Eref 2350569
	 bFFP 23

In [12]:
# concatenate environmental output by each env, combine chains into single df
dflst = {}
for chain,envs in envlist.items():
    for env,dfs in nb(envs.items(), desc=chain):
        if env not in dflst:
            dflst[env] = []
        df = pd.concat(dfs)
        df.columns = [f"{chain}-{col}" for col in df.columns]
        dflst[env].append(df)
# combine chains for each env - should have done this in parallel
envdfs = {}
for env,lst in nb(dflst.items(), desc='combo'):
    assert len(lst) == 5
    df = pd.merge(lst[0], lst[1], left_index=True, right_index=True)
    df2 = pd.merge(lst[2], lst[3], left_index=True, right_index=True)
    df3 = pd.merge(df, df2, left_index=True, right_index=True)
    envdfs[env] = pd.merge(df3, lst[4], left_index=True, right_index=True)
    
    
envdfs[env].head()           

combo: 100%|██████████| 19/19 [03:15<00:00, 10.31s/it]


,chain_1-COVARIABLE,chain_1-MRK,chain_1-M_Pearson,chain_1-SD_Pearson,chain_1-BF(dB),chain_1-Beta_is,chain_1-SD_Beta_is,chain_1-eBPis,chain_1-env,chain_2-COVARIABLE,chain_2-MRK,chain_2-M_Pearson,chain_2-SD_Pearson,chain_2-BF(dB),chain_2-Beta_is,chain_2-SD_Beta_is,chain_2-eBPis,chain_2-env,chain_3-COVARIABLE,chain_3-MRK,chain_3-M_Pearson,chain_3-SD_Pearson,chain_3-BF(dB),chain_3-Beta_is,chain_3-SD_Beta_is,chain_3-eBPis,chain_3-env,chain_4-COVARIABLE,chain_4-MRK,chain_4-M_Pearson,chain_4-SD_Pearson,chain_4-BF(dB),chain_4-Beta_is,chain_4-SD_Beta_is,chain_4-eBPis,chain_4-env,chain_5-COVARIABLE,chain_5-MRK,chain_5-M_Pearson,chain_5-SD_Pearson,chain_5-BF(dB),chain_5-Beta_is,chain_5-SD_Beta_is,chain_5-eBPis,chain_5-env
jcf7190000000000-77738,1,1,0.019481,0.143261,-9.818379,0.001109,0.009161,0.044014,MAT,1,1,0.016763,0.139889,-9.763548,0.001180,0.010172,0.042086,MAT,1,1,0.040410,0.134280,-8.941268,0.002665,0.008408,0.124187,MAT,1,1,0.011143,0.142846,-8.294361,-0.000112,0.011705,0.003325,MAT,1,1,0.040552,0.146463,-8.970313,0.003243,0.010373,0.122298,MAT
jcf7190000000000-77764,1,2,0.067087,0.131969,-4.168396,0.005074,0.009358,0.230858,MAT,1,2,0.051400,0.137479,-9.268755,0.004405,0.009922,0.182391,MAT,1,2,0.053341,0.145438,-8.886951,0.004971,0.011208,0.182166,MAT,1,2,0.053708,0.136611,-8.497892,0.004676,0.010306,0.187071,MAT,1,2,0.048398,0.123070,-9.735437,0.004928,0.009452,0.220294,MAT
jcf7190000000004-54418,1,3,-0.010818,0.142644,-11.520927,-0.001558,0.005309,0.113974,MAT,1,3,-0.005682,0.139501,-12.805761,-0.000997,0.005383,0.069020,MAT,1,3,-0.008921,0.141621,-11.531844,-0.001592,0.005944,0.103033,MAT,1,3,-0.005990,0.141390,-12.572324,-0.001247,0.005522,0.085514,MAT,1,3,-0.011867,0.146292,-11.622625,-0.001574,0.005674,0.107115,MAT
jcf7190000000004-54508,1,4,-0.046764,0.117936,-11.884691,-0.001756,0.006781,0.099268,MAT,1,4,-0.066645,0.110145,-11.360039,-0.006334,0.009368,0.301908,MAT,1,4,-0.046778,0.113096,-12.602799,-0.003065,0.005446,0.241436,MAT,1,4,-0.056804,0.116794,-11.745844,-0.003762,0.006513,0.249077,MAT,1,4,-0.069587,0.114737,-12.324292,-0.003220,0.005251,0.267834,MAT
jcf7190000000004-54540,1,5,0.021122,0.128426,-11.381520,0.000681,0.007170,0.034198,MAT,1,5,0.038666,0.132323,-10.957807,0.001610,0.007318,0.083071,MAT,1,5,0.021904,0.129940,-12.165769,0.000721,0.007136,0.036464,MAT,1,5,0.025468,0.125963,-12.216768,0.000900,0.006982,0.046970,MAT,1,5,0.013424,0.137231,-11.306552,0.000024,0.008280,0.001017,MAT


In [14]:
# how many SNPs should be in each env file?
rows = 0
for gfile in nb(fs(gfile_dir, pattern='noheader')):
    df = pd.read_table(gfile, usecols=[0], header=None)
    rows += nrow(df)
rows

100%|██████████| 1022/1022 [00:05<00:00, 183.79it/s]


2350569

In [15]:
for env in envdfs:
    print(ColorText(env).bold())
    print('\t',env, nrow(envdfs[env]))
    if nrow(envdfs[env]) != rows:
        print(ColorText('CRAAAAAAAAAP').fail().bold())

DD_0
	 DD_0 2350569
MCMT
	 MCMT 2350569
EMT
	 EMT 2350569
FFP
	 FFP 2350569
Eref
	 Eref 2350569
bFFP
	 bFFP 2350569
PAS
	 PAS 2350569
TD
	 TD 2350569
AHM
	 AHM 2350569
EXT
	 EXT 2350569
eFFP
	 eFFP 2350569
SHM
	 SHM 2350569
NFFD
	 NFFD 2350569
MAP
	 MAP 2350569
MSP
	 MSP 2350569
CMD
	 CMD 2350569
MWMT
	 MWMT 2350569
DD5
	 DD5 2350569
MAT
	 MAT 2350569


In [16]:
# save temp state of envdfs
finaldir = makedir(op.join(baydir, 'final_results'))
pkldump(envdfs, op.join(finaldir, 'envdfs_before_ranking.pkl'))

In [17]:
finaldir

'/lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/coastal_variety/03_maf-p05_RD-recalculated_FDC/baypass/final_results'

In [18]:
# release some RAM
del dflst, chaindfs, df, df2, df3, dfs, envlist, jobs

# make columns for ranking

In [3]:
lview,dview = get_client()

56 56


In [4]:
finaldir = op.join(baydir, 'final_results')
envdfs = pklload(op.join(finaldir, 'envdfs_before_ranking.pkl'))
len(envdfs)

19

In [5]:
def rank_baypass(df):
    """Create columns to rank loci within and across chains."""
    import pandas
    import tqdm
    import math
    chains = ['chain_1', 'chain_2', 'chain_3', 'chain_4', 'chain_5']

    # rank per chain based on chain BF
    for chain in tqdm.tqdm(chains, desc='chains'):
        col = f'{chain}-BF(dB)'
        ranked = dict((locus,rank+1) for (rank,locus) in enumerate(df[col].sort_values(ascending=False).index))
        df[f'rank_{chain}-BF(dB)'] = [ranked[locus] for locus in df.index]

    # mean BF
    print('getting mean BF')
    bfcols = [col for col in df if 'BF(dB)' in col and 'rank' not in col]
    print(bfcols)
    assert len(bfcols) == 5
    df['mean_BF(dB)'] = df[bfcols].mean(axis=1)

    # rank of mean BF
    print('ranking mean BF')
    ranked = dict((locus,rank+1) for (rank,locus) in enumerate(df['mean_BF(dB)'].sort_values(ascending=False).index))
    df['rank_mean_BF(dB)'] = [ranked[locus] for locus in df.index]
    
    # bool column, enter True if BF >= 20 for >=3 chains
    # BF column is in dB units, value output by baypass = 10*log10(BF)
    print('calculating bool column')
    df['BF(dB)_gte20_for-gte3chains'] = (df[bfcols] >= 10*math.log10(20)).sum(axis=1) > 3
    print(sum((df[bfcols] > 10*math.log10(20)).sum(axis=1) > 3))

    return df

In [30]:
# test function
out = rank_baypass(envdfs[env])

chains: 100%|██████████| 5/5 [00:23<00:00,  4.69s/it]


getting mean BF
['chain_1-BF(dB)', 'chain_2-BF(dB)', 'chain_3-BF(dB)', 'chain_4-BF(dB)', 'chain_5-BF(dB)']
ranking mean BF
calculating bool column
309


In [31]:
sum(out['BF(dB)_gte20_for-gte3chains'])

309

In [6]:
# call rank_baypass() in parallel for each environment
jobs = []
for env,df in envdfs.items():
    bfcols = [col for col in df.columns if 'BF(dB)' in col]
    assert len(bfcols) == 5  # num chains
    jobs.append(lview.apply_async(rank_baypass, df[bfcols]))
watch_async(jobs)

19
19


In [19]:
# get the returns from the engines
new_envdfs = {}
for i,env in enumerate(envdfs):
    new_envdfs[env] = jobs[i].r
new_envdfs[env].head()    

,chain_1-BF(dB),chain_2-BF(dB),chain_3-BF(dB),chain_4-BF(dB),chain_5-BF(dB),rank_chain_1-BF(dB),rank_chain_2-BF(dB),rank_chain_3-BF(dB),rank_chain_4-BF(dB),rank_chain_5-BF(dB),mean_BF(dB),rank_mean_BF(dB),BF(dB)_gte20_for-gte3chains
jcf7190000000000-77738,-9.818379,-9.763548,-8.941268,-8.294361,-8.970313,674303,660606,487259,390696,492129,-9.157574,512312,False
jcf7190000000000-77764,-4.168396,-9.268755,-8.886951,-8.497892,-9.735437,112289,548242,477880,418517,653159,-8.111486,343925,False
jcf7190000000004-54418,-11.520927,-12.805761,-11.531844,-12.572324,-11.622625,1305721,1795502,1309915,1720478,1349973,-12.010696,1575714,False
jcf7190000000004-54508,-11.884691,-11.360039,-12.602799,-11.745844,-12.324292,1460397,1235529,1729590,1403391,1633079,-11.983533,1564772,False
jcf7190000000004-54540,-11.381520,-10.957807,-12.165769,-12.216768,-11.306552,1245463,1063397,1572648,1593009,1212267,-11.605683,1405070,False


In [20]:
# returns from the function or only a subset of the original columns, make sure they match up (they should)
print(env)
envdfs[env].head()

MAT


,chain_1-COVARIABLE,chain_1-MRK,chain_1-M_Pearson,chain_1-SD_Pearson,chain_1-BF(dB),chain_1-Beta_is,chain_1-SD_Beta_is,chain_1-eBPis,chain_1-env,chain_2-COVARIABLE,chain_2-MRK,chain_2-M_Pearson,chain_2-SD_Pearson,chain_2-BF(dB),chain_2-Beta_is,chain_2-SD_Beta_is,chain_2-eBPis,chain_2-env,chain_3-COVARIABLE,chain_3-MRK,chain_3-M_Pearson,chain_3-SD_Pearson,chain_3-BF(dB),chain_3-Beta_is,chain_3-SD_Beta_is,chain_3-eBPis,chain_3-env,chain_4-COVARIABLE,chain_4-MRK,chain_4-M_Pearson,chain_4-SD_Pearson,chain_4-BF(dB),chain_4-Beta_is,chain_4-SD_Beta_is,chain_4-eBPis,chain_4-env,chain_5-COVARIABLE,chain_5-MRK,chain_5-M_Pearson,chain_5-SD_Pearson,chain_5-BF(dB),chain_5-Beta_is,chain_5-SD_Beta_is,chain_5-eBPis,chain_5-env
jcf7190000000000-77738,1,1,0.019481,0.143261,-9.818379,0.001109,0.009161,0.044014,MAT,1,1,0.016763,0.139889,-9.763548,0.001180,0.010172,0.042086,MAT,1,1,0.040410,0.134280,-8.941268,0.002665,0.008408,0.124187,MAT,1,1,0.011143,0.142846,-8.294361,-0.000112,0.011705,0.003325,MAT,1,1,0.040552,0.146463,-8.970313,0.003243,0.010373,0.122298,MAT
jcf7190000000000-77764,1,2,0.067087,0.131969,-4.168396,0.005074,0.009358,0.230858,MAT,1,2,0.051400,0.137479,-9.268755,0.004405,0.009922,0.182391,MAT,1,2,0.053341,0.145438,-8.886951,0.004971,0.011208,0.182166,MAT,1,2,0.053708,0.136611,-8.497892,0.004676,0.010306,0.187071,MAT,1,2,0.048398,0.123070,-9.735437,0.004928,0.009452,0.220294,MAT
jcf7190000000004-54418,1,3,-0.010818,0.142644,-11.520927,-0.001558,0.005309,0.113974,MAT,1,3,-0.005682,0.139501,-12.805761,-0.000997,0.005383,0.069020,MAT,1,3,-0.008921,0.141621,-11.531844,-0.001592,0.005944,0.103033,MAT,1,3,-0.005990,0.141390,-12.572324,-0.001247,0.005522,0.085514,MAT,1,3,-0.011867,0.146292,-11.622625,-0.001574,0.005674,0.107115,MAT
jcf7190000000004-54508,1,4,-0.046764,0.117936,-11.884691,-0.001756,0.006781,0.099268,MAT,1,4,-0.066645,0.110145,-11.360039,-0.006334,0.009368,0.301908,MAT,1,4,-0.046778,0.113096,-12.602799,-0.003065,0.005446,0.241436,MAT,1,4,-0.056804,0.116794,-11.745844,-0.003762,0.006513,0.249077,MAT,1,4,-0.069587,0.114737,-12.324292,-0.003220,0.005251,0.267834,MAT
jcf7190000000004-54540,1,5,0.021122,0.128426,-11.381520,0.000681,0.007170,0.034198,MAT,1,5,0.038666,0.132323,-10.957807,0.001610,0.007318,0.083071,MAT,1,5,0.021904,0.129940,-12.165769,0.000721,0.007136,0.036464,MAT,1,5,0.025468,0.125963,-12.216768,0.000900,0.006982,0.046970,MAT,1,5,0.013424,0.137231,-11.306552,0.000024,0.008280,0.001017,MAT


In [22]:
# add in the new columns
for env,df in envdfs.items():
    update([env])
    newcols = [col for col in new_envdfs[env].columns if col not in df.columns]
    envdfs[env] = pd.merge(df, new_envdfs[env][newcols], right_index=True, left_index=True)
envdfs[env].head()

MAT


,chain_1-COVARIABLE,chain_1-MRK,chain_1-M_Pearson,chain_1-SD_Pearson,chain_1-BF(dB),chain_1-Beta_is,chain_1-SD_Beta_is,chain_1-eBPis,chain_1-env,chain_2-COVARIABLE,chain_2-MRK,chain_2-M_Pearson,chain_2-SD_Pearson,chain_2-BF(dB),chain_2-Beta_is,chain_2-SD_Beta_is,chain_2-eBPis,chain_2-env,chain_3-COVARIABLE,chain_3-MRK,chain_3-M_Pearson,chain_3-SD_Pearson,chain_3-BF(dB),chain_3-Beta_is,chain_3-SD_Beta_is,chain_3-eBPis,chain_3-env,chain_4-COVARIABLE,chain_4-MRK,chain_4-M_Pearson,chain_4-SD_Pearson,chain_4-BF(dB),chain_4-Beta_is,chain_4-SD_Beta_is,chain_4-eBPis,chain_4-env,chain_5-COVARIABLE,chain_5-MRK,chain_5-M_Pearson,chain_5-SD_Pearson,chain_5-BF(dB),chain_5-Beta_is,chain_5-SD_Beta_is,chain_5-eBPis,chain_5-env,rank_chain_1-BF(dB),rank_chain_2-BF(dB),rank_chain_3-BF(dB),rank_chain_4-BF(dB),rank_chain_5-BF(dB),mean_BF(dB),rank_mean_BF(dB),BF(dB)_gte20_for-gte3chains
jcf7190000000000-77738,1,1,0.019481,0.143261,-9.818379,0.001109,0.009161,0.044014,MAT,1,1,0.016763,0.139889,-9.763548,0.001180,0.010172,0.042086,MAT,1,1,0.040410,0.134280,-8.941268,0.002665,0.008408,0.124187,MAT,1,1,0.011143,0.142846,-8.294361,-0.000112,0.011705,0.003325,MAT,1,1,0.040552,0.146463,-8.970313,0.003243,0.010373,0.122298,MAT,674303,660606,487259,390696,492129,-9.157574,512312,False
jcf7190000000000-77764,1,2,0.067087,0.131969,-4.168396,0.005074,0.009358,0.230858,MAT,1,2,0.051400,0.137479,-9.268755,0.004405,0.009922,0.182391,MAT,1,2,0.053341,0.145438,-8.886951,0.004971,0.011208,0.182166,MAT,1,2,0.053708,0.136611,-8.497892,0.004676,0.010306,0.187071,MAT,1,2,0.048398,0.123070,-9.735437,0.004928,0.009452,0.220294,MAT,112289,548242,477880,418517,653159,-8.111486,343925,False
jcf7190000000004-54418,1,3,-0.010818,0.142644,-11.520927,-0.001558,0.005309,0.113974,MAT,1,3,-0.005682,0.139501,-12.805761,-0.000997,0.005383,0.069020,MAT,1,3,-0.008921,0.141621,-11.531844,-0.001592,0.005944,0.103033,MAT,1,3,-0.005990,0.141390,-12.572324,-0.001247,0.005522,0.085514,MAT,1,3,-0.011867,0.146292,-11.622625,-0.001574,0.005674,0.107115,MAT,1305721,1795502,1309915,1720478,1349973,-12.010696,1575714,False
jcf7190000000004-54508,1,4,-0.046764,0.117936,-11.884691,-0.001756,0.006781,0.099268,MAT,1,4,-0.066645,0.110145,-11.360039,-0.006334,0.009368,0.301908,MAT,1,4,-0.046778,0.113096,-12.602799,-0.003065,0.005446,0.241436,MAT,1,4,-0.056804,0.116794,-11.745844,-0.003762,0.006513,0.249077,MAT,1,4,-0.069587,0.114737,-12.324292,-0.003220,0.005251,0.267834,MAT,1460397,1235529,1729590,1403391,1633079,-11.983533,1564772,False
jcf7190000000004-54540,1,5,0.021122,0.128426,-11.381520,0.000681,0.007170,0.034198,MAT,1,5,0.038666,0.132323,-10.957807,0.001610,0.007318,0.083071,MAT,1,5,0.021904,0.129940,-12.165769,0.000721,0.007136,0.036464,MAT,1,5,0.025468,0.125963,-12.216768,0.000900,0.006982,0.046970,MAT,1,5,0.013424,0.137231,-11.306552,0.000024,0.008280,0.001017,MAT,1245463,1063397,1572648,1593009,1212267,-11.605683,1405070,False


In [8]:
def write_file(df, file, sep='\t', index=False):
    import pandas
    df['locus'] = df.index.tolist()
    df = df[['locus'] + [col for col in df.columns if col != 'locus']]
    df.to_csv(file, sep=sep, index=index)
    return file

In [25]:
finaldir

'/lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/coastal_variety/03_maf-p05_RD-recalculated_FDC/baypass/final_results'

In [26]:
ls(finaldir)

['envdfs_before_ranking.pkl']

In [9]:
# save files
jobs = []
for env,df in envdfs.items():
    file = op.join(finaldir, f'coastal_maf-p05_RD-recalculated_ADP-lt-1000_{env}.txt')
    jobs.append(lview.apply_async(write_file, *(df, file)))
watch_async(jobs, phase='final')

final
19
19


In [28]:
# save pkl for easy loading
pkldump(envdfs, op.join(finaldir, 'envdfs_after_ranking.pkl'))

In [30]:
op.join(finaldir, 'envdfs_after_ranking.pkl')

'/lu213/brandon.lind/projects/pool_seq/DF_datasets/DF_pooled_GEA/DF_pooled/snpsANDindels/coastal_variety/03_maf-p05_RD-recalculated_FDC/baypass/final_results/envdfs_after_ranking.pkl'

#### how many snps per env with mean BF > 20

In [12]:
print(ColorText('Coastal\n').bold().green())
print('Env   Consistent  Mean')
for env in sorted(envdfs.keys()):
    df = envdfs[env]
    print(env,'\t',sum(df['BF(dB)_gte20_for-gte3chains']),'\t  ',sum(df['mean_BF(dB)']>=10*math.log10(20)))

Coastal

Env   Consistent  Mean
AHM 	 1213 	   2123
CMD 	 985 	   1767
DD5 	 284 	   567
DD_0 	 360 	   642
EMT 	 287 	   528
EXT 	 456 	   924
Eref 	 1116 	   2097
FFP 	 224 	   445
MAP 	 705 	   1162
MAT 	 309 	   609
MCMT 	 328 	   579
MSP 	 784 	   1450
MWMT 	 358 	   720
NFFD 	 257 	   489
PAS 	 716 	   1189
SHM 	 1149 	   2272
TD 	 546 	   891
bFFP 	 249 	   467
eFFP 	 216 	   411
